In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

Utilise your web scraping skills to gather information about three German cities – Berlin, Hamburg, and Munich – from Wikipedia. You will start by extracting the population of each city and then expand the scope of your data gathering to include latitude and longitude, country, and possibly other relevant details.

1. Population Scraping

  1.1. Begin by scraping the population of each city from their respective Wikipedia pages:

 - Berlin: https://en.wikipedia.org/wiki/Berlin
 - Hamburg: https://en.wikipedia.org/wiki/Hamburg
 - Munich: https://en.wikipedia.org/wiki/Munich

  1.2. Once you have scrapped the population of each city, reflect on the similarities and patterns in accessing the population data across the three pages. Also, analyse the URLs to identify any commonalities. Make a loop that executes once but simultaneously retrieves the population for all three cities.

2. Data Organisation

  Utilise pandas DataFrame to effectively store the extracted population data. Ensure the data is clean and properly formatted. Remove any unnecessary characters or symbols and ensure the column data types are accurate.

3. Further Enhancement

  3.1. Expand the scope of your data gathering by extracting other relevant information for each city:

 - Latitude and longitude
 - Country of location

  3.2. Create a function from the loop and DataFrame to encapsulate the scraping process. This function can be used repeatedly to fetch updated data whenever necessary. It should return a clean, properly formatted DataFrame.

Getting the coordinates of only one city (Berlin)

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
city_name="Berlin"
url = "https://en.wikipedia.org/wiki/"+city_name  # Website URL
response = requests.get(url) # Get the response from the URL
soup_city = bs(response.content, 'html.parser') # Use Beautiful Soup to parse the HTML 

latitude=soup_city.find(class_="latitude").get_text() # Get the latitude 
longitude=soup_city.find(class_="longitude").get_text() # Get the longitude 
print(latitude, longitude)

52°31′12″N 13°24′18″E


First solution using lists

In [3]:
cities = [
    "Berlin",
    "Hamburg",
    "Munich",
    "Stuttgart"
]
country_code=[]
latitude=[]
longitude=[]
population=[]
year=[]

In [4]:
def get_city_info_list(cities):
  for city_name in cities:
    url = "https://en.wikipedia.org/wiki/"+city_name
    response = requests.get(url)
    soup_city = bs(response.content, 'html.parser')
    #Country
    country=soup_city.select('td.infobox-data')[0].text
    #Country code
    match country:
      case "Germany":code="DE"
      case "France":code="FR"
    country_code.append(code)
    #Coordinates
    latitude.append(soup_city.find(class_="latitude").get_text())
    longitude.append(soup_city.find(class_="longitude").get_text())
    #Population
    population.append(soup_city.find('table', class_='vcard').find(string="Population").find_next("td").text)
    #Year
    year.append(soup_city.find('table', class_='vcard').find(string="Population").find_next("div").text[2:6])

  cities_df = pd.DataFrame(
    {"city": cities,
     "country_code": country_code,
     "latitude": latitude,
     "longitude": longitude,
     "population": population,
     "year_data_retrieved": year,
    })
  return cities_df

In [5]:
cities_df_list=get_city_info_list(cities)
cities_df_list

,city,country_code,latitude,longitude,population,year_data_retrieved
0,Berlin,DE,52°31′12″N,13°24′18″E,"3,576,873",2024
1,Hamburg,DE,53°33′N,10°00′E,"1,945,532",2022
2,Munich,DE,48°08′15″N,11°34′30″E,"1,512,491",2022
3,Stuttgart,DE,48°46′39″N,09°10′48″E,"626,275",2021


Second solution using dictionaries

In [6]:
cities = [
    "Berlin",
    "Hamburg",
    "Munich",
    "Stuttgart"
]

In [7]:
def get_city_info_dictionary(cities):

  city_dic = {"city": [],
    "country_code": [],
    "latitude": [],
    "longitude": [],
    "population": [],
    "year_data_retrieved": []
    }

  for city_name in cities:
    url = "https://en.wikipedia.org/wiki/"+city_name
    response = requests.get(url)
    soup_city = bs(response.content, 'html.parser')
    city_dic["city"].append(city_name)
    #Country
    country=soup_city.select('td.infobox-data')[0].text
    match country:
      case "Germany":code="DE"
      case "France":code="FR"
    city_dic["country_code"].append(code)
    #Coordinates
    city_dic["latitude"].append(soup_city.find(class_="latitude").get_text())
    city_dic["longitude"].append(soup_city.find(class_="longitude").get_text())
    #Population
    city_dic["population"].append(soup_city.find('table', class_='vcard').find(string="Population").find_next("td").text)
    #Year
    city_dic["year_data_retrieved"].append(soup_city.find('table', class_='vcard').find(string="Population").find_next("div").text[2:6])

    cities_df = pd.DataFrame(city_dic)
  return cities_df

In [8]:
cities_df_dic=get_city_info_dictionary(cities)
cities_df_dic

,city,country_code,latitude,longitude,population,year_data_retrieved
0,Berlin,DE,52°31′12″N,13°24′18″E,"3,576,873",2024
1,Hamburg,DE,53°33′N,10°00′E,"1,945,532",2022
2,Munich,DE,48°08′15″N,11°34′30″E,"1,512,491",2022
3,Stuttgart,DE,48°46′39″N,09°10′48″E,"626,275",2021


Data cleaning

In [9]:
cities_df_dic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   city                 4 non-null      object
 1   country_code         4 non-null      object
 2   latitude             4 non-null      object
 3   longitude            4 non-null      object
 4   population           4 non-null      object
 5   year_data_retrieved  4 non-null      object
dtypes: object(6)
memory usage: 324.0+ bytes


In [10]:
cities_df_dic.population = cities_df_dic.population.apply(lambda x : x.replace(',',''))

In [11]:
# Convert data columns to the right types
cities_df_dic['population'] = cities_df_dic['population'].astype(int)
cities_df_dic['year_data_retrieved'] = cities_df_dic['year_data_retrieved'].astype(int)

# Check the changes took place and are correct
cities_df_dic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   city                 4 non-null      object
 1   country_code         4 non-null      object
 2   latitude             4 non-null      object
 3   longitude            4 non-null      object
 4   population           4 non-null      int64 
 5   year_data_retrieved  4 non-null      int64 
dtypes: int64(2), object(4)
memory usage: 324.0+ bytes


In [12]:
cities_df_dic

,city,country_code,latitude,longitude,population,year_data_retrieved
0,Berlin,DE,52°31′12″N,13°24′18″E,3576873,2024
1,Hamburg,DE,53°33′N,10°00′E,1945532,2022
2,Munich,DE,48°08′15″N,11°34′30″E,1512491,2022
3,Stuttgart,DE,48°46′39″N,09°10′48″E,626275,2021


Preparing tables to send them to SQL

In [13]:
cities_country_df=cities_df_dic.loc[:,["city","country_code"]]
cities_country_df

,city,country_code
0,Berlin,DE
1,Hamburg,DE
2,Munich,DE
3,Stuttgart,DE


In [14]:
cities_infos_df=cities_df_dic.loc[:,["city","latitude","longitude","population","year_data_retrieved"]]
cities_infos_df

,city,latitude,longitude,population,year_data_retrieved
0,Berlin,52°31′12″N,13°24′18″E,3576873,2024
1,Hamburg,53°33′N,10°00′E,1945532,2022
2,Munich,48°08′15″N,11°34′30″E,1512491,2022
3,Stuttgart,48°46′39″N,09°10′48″E,626275,2021


Connect to SQL

In [15]:
# install if needed
# !pip install sqlalchemy

In [16]:
from my_passwords import my_password

In [17]:
# Connect to the database
schema = "gans"
host = "127.0.0.1"
user = "root"
password = my_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [18]:
# Send data of cities_country_df to "cities" table
cities_country_df.to_sql('cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)

4

In [19]:
# Get the table "cities"
cities_from_sql = pd.read_sql("cities", con=connection_string)
cities_from_sql

,city_id,city,country_code
0,1,Berlin,DE
1,2,Hamburg,DE
2,3,Munich,DE
3,4,Stuttgart,DE


In [20]:
# Merge city_infos_df with cities_from_sql to get the city ID
city_infos_df = cities_infos_df.merge(cities_from_sql,
                                   on = "city",
                                   how="left")

city_infos_df

,city,latitude,longitude,population,year_data_retrieved,city_id,country_code
0,Berlin,52°31′12″N,13°24′18″E,3576873,2024,1,DE
1,Hamburg,53°33′N,10°00′E,1945532,2022,2,DE
2,Munich,48°08′15″N,11°34′30″E,1512491,2022,3,DE
3,Stuttgart,48°46′39″N,09°10′48″E,626275,2021,4,DE


In [21]:
# Drop the two columns "city" and "country_code"
city_infos_df = city_infos_df.drop(columns=["city","country_code"])
city_infos_df

,latitude,longitude,population,year_data_retrieved,city_id
0,52°31′12″N,13°24′18″E,3576873,2024,1
1,53°33′N,10°00′E,1945532,2022,2
2,48°08′15″N,11°34′30″E,1512491,2022,3
3,48°46′39″N,09°10′48″E,626275,2021,4


In [22]:
# Change the order of the columns
city_infos_df=city_infos_df[["city_id","latitude","longitude","population","year_data_retrieved"]]
city_infos_df

,city_id,latitude,longitude,population,year_data_retrieved
0,1,52°31′12″N,13°24′18″E,3576873,2024
1,2,53°33′N,10°00′E,1945532,2022
2,3,48°08′15″N,11°34′30″E,1512491,2022
3,4,48°46′39″N,09°10′48″E,626275,2021


In [23]:
# Send data of city_infos_df to "city_infos" table
city_infos_df.to_sql('city_infos',
                if_exists='append',
                con=connection_string,
                index=False)

4

In [24]:
# Get the table "city_infos"
city_infos_from_sql = pd.read_sql("city_infos", con=connection_string)
city_infos_from_sql

,city_id,latitude,longitude,population,year_data_retrieved
0,1,52°31′12″N,13°24′18″E,3576873,2024
1,2,53°33′N,10°00′E,1945532,2022
2,3,48°08′15″N,11°34′30″E,1512491,2022
3,4,48°46′39″N,09°10′48″E,626275,2021
